In [1]:
%matplotlib qt

In [2]:
import os, pyxdf, json, yaml
import mne
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from mne.time_frequency import psd_welch, tfr_morlet, tfr_multitaper
# from multitaper_spectrogram_python import multitaper_spectrogram
from mne.decoding import Scaler, Vectorizer

from sklearn.pipeline import make_pipeline
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import RepeatedStratifiedKFold, HalvingGridSearchCV

from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier

### Settings

In [3]:
config_file = 'config_MI-hands.yaml'

with open(config_file) as f:
    config = yaml.load(f.read(), Loader=yaml.Loader)
    print(config)
    locals().update(config)

lslDir = os.path.join(os.path.expanduser('~'), 'Documents\CurrentStudy')

{'subject': 'P00J', 'session': 1, 'task': 'MI-hands', 'eeg_stream_type': 'EXG', 'markers_stream_type': 'Marker', 'default_ch_names': ['Fz', 'FC1', 'FC2', 'C3', 'Cz', 'C4', 'CP1', 'CP2', 'P3', 'Pz', 'P4', 'PO3', 'PO4', 'O1', 'Oz', 'O2'], 'montage_file': 'openbci_montage.elc', 'event_dict': {'rest': 0, 'MI/hands': 1}, 'rest_duration': 5, 'task_duration': 5, 'tmin': 0, 'tmax': 5, 'plotGraphs': False, 'scalings': {'eeg': 0.0002}, 'plot_duration': 10, 'bp_l_freq': 1.0, 'bp_h_freq': 40.0, 'performICA': False, 'features': 'tfr', 'tfr_type': 'morlet', 'baseline': (0.0, 0.1), 'vmin': -1.0, 'vmax': 1.0}


### Find LSL Files

In [4]:
# Find files
xdf_files = []
hasSubject = subject!=''
hasSession = session!=''
hasTask = task!=''
for root, dir, files in os.walk(lslDir):
    for file in files:
        validFile = True
        if hasSubject:
            validFile = validFile and (('sub-'+subject) in file)
        if hasSession:
            validFile = validFile and (('ses-S' + str(session).zfill(3)) in file)
        if hasTask:
            validFile = validFile and (('task-' + task) in file)
        validFile = validFile and file.endswith('.xdf')
        if validFile:
            print(file)
            matchingFile = os.path.join(root, file)
            xdf_files.append(matchingFile)

if len(xdf_files) == 0:
    print('No files found')

sub-P00J_ses-S001_task-MI-hands_run-001_eeg.xdf
sub-P00J_ses-S001_task-MI-hands_run-002_eeg.xdf
sub-P00J_ses-S001_task-MI-hands_run-003_eeg.xdf
sub-P00J_ses-S001_task-MI-hands_run-004_eeg.xdf
sub-P00J_ses-S001_task-MI-hands_run-005_eeg.xdf


In [5]:
# Parse streams
eeg_stream, marker_stream = [], []

print('Parsing streams')
for xdf_file in xdf_files:
    streams, header = pyxdf.load_xdf(xdf_file)
    for i in range(len(streams)):
        if streams[i]['info']['type'][0] == eeg_stream_type:
            print("Found %s stream in %s" % (eeg_stream_type, os.path.basename(xdf_file)))
            eeg_stream.append(streams[i])
        elif streams[i]['info']['type'][0] == markers_stream_type:
            print("Found %s stream in %s" % (markers_stream_type, os.path.basename(xdf_file)))
            marker_stream.append(streams[i])
del streams, header

Parsing streams
Found Marker stream in sub-P00J_ses-S001_task-MI-hands_run-001_eeg.xdf
Found EXG stream in sub-P00J_ses-S001_task-MI-hands_run-001_eeg.xdf
Found EXG stream in sub-P00J_ses-S001_task-MI-hands_run-002_eeg.xdf
Found Marker stream in sub-P00J_ses-S001_task-MI-hands_run-002_eeg.xdf
Found EXG stream in sub-P00J_ses-S001_task-MI-hands_run-003_eeg.xdf
Found Marker stream in sub-P00J_ses-S001_task-MI-hands_run-003_eeg.xdf
Found EXG stream in sub-P00J_ses-S001_task-MI-hands_run-004_eeg.xdf
Found Marker stream in sub-P00J_ses-S001_task-MI-hands_run-004_eeg.xdf
Found Marker stream in sub-P00J_ses-S001_task-MI-hands_run-005_eeg.xdf
Found EXG stream in sub-P00J_ses-S001_task-MI-hands_run-005_eeg.xdf


### Extract EEG and Marker data

In [6]:
# Extract EEG Info
print("Extracting EEG info...")

ch_names = []
if eeg_stream[0]['info']['desc'][0]:
    print("EEG channel names found")
    for i in range(len(eeg_stream[0]['info']['desc'][0]['channels'][0]['channel'])):
        ch_names.append(eeg_stream[0]['info']['desc'][0]['channels'][0]['channel'][i]['label'][0])
else:
    ch_names = default_ch_names
print('Channels: ', ch_names)

sfreq = float(eeg_stream[0]['info']['nominal_srate'][0])
print('Sampling frequency: ', sfreq)

# Create MNE info object
eeg_info = mne.create_info(ch_names, sfreq, ch_types='eeg')

Extracting EEG info...
Channels:  ['Fz', 'FC1', 'FC2', 'C3', 'Cz', 'C4', 'CP1', 'CP2', 'P3', 'Pz', 'P4', 'PO3', 'PO4', 'O1', 'Oz', 'O2']
Sampling frequency:  125.0


In [7]:
# Setup Montage
montage = mne.channels.read_custom_montage(montage_file)
# montage.plot()

In [8]:
# Get all EEG data
eeg_raw_list = []

for n in range(len(eeg_stream)):
    # Create MNE Raw object
    eeg_data = np.transpose(eeg_stream[n]['time_series'])
    eeg_data = eeg_data / 1e6
    print(eeg_data.shape)
    eeg_raw = mne.io.RawArray(eeg_data, eeg_info)
    
    # Set montage
    eeg_raw = eeg_raw.set_montage(montage)

    # Add annotations
    onset, duration, description = [], [], []
    current_target = -1
    current_flash = -1
    for i in range(len(marker_stream[n]['time_series'])):
        if 'MI' in task:
            if ('rest' in marker_stream[n]['time_series'][i][0]):
                onset.append(marker_stream[n]['time_stamps'][i] - eeg_stream[n]['time_stamps'][0])
                duration.append(rest_duration)
                description.append(marker_stream[n]['time_series'][i][0])
            elif ('task' in marker_stream[n]['time_series'][i][0]):
                onset.append(marker_stream[n]['time_stamps'][i] - eeg_stream[n]['time_stamps'][0])
                duration.append(task_duration)
                description.append(marker_stream[n]['time_series'][i][0].replace('task_', '').replace('-','/'))
        elif 'P300' in task:
            if('target' in marker_stream[n]['time_series'][i][0]):
                current_target = json.loads(marker_stream[n]['time_series'][i][0])['target']
            elif('flash' in marker_stream[n]['time_series'][i][0]):
                current_flash = json.loads(marker_stream[n]['time_series'][i][0])['flash']
                onset.append(marker_stream[n]['time_stamps'][i] - eeg_stream[n]['time_stamps'][0])
                duration.append(task_duration)
                description.append("target" if current_flash == current_target else "nontarget")
    annotations = mne.Annotations(onset, duration, description)
    eeg_raw = eeg_raw.set_annotations(annotations)
    
    # Create list of raw objects
    eeg_raw_list.append(eeg_raw)

(16, 21034)
Creating RawArray with float64 data, n_channels=16, n_times=21034
    Range : 0 ... 21033 =      0.000 ...   168.264 secs
Ready.
(16, 20911)
Creating RawArray with float64 data, n_channels=16, n_times=20911
    Range : 0 ... 20910 =      0.000 ...   167.280 secs
Ready.
(16, 20765)
Creating RawArray with float64 data, n_channels=16, n_times=20765
    Range : 0 ... 20764 =      0.000 ...   166.112 secs
Ready.
(16, 20644)
Creating RawArray with float64 data, n_channels=16, n_times=20644
    Range : 0 ... 20643 =      0.000 ...   165.144 secs
Ready.
(16, 20760)
Creating RawArray with float64 data, n_channels=16, n_times=20760
    Range : 0 ... 20759 =      0.000 ...   166.072 secs
Ready.


In [9]:
# Concatenate raw objects
raw = mne.concatenate_raws(eeg_raw_list)
raw

<RawArray | 16 x 104114 (832.9 s), ~12.7 MB, data loaded>

### Pre-processing

In [10]:
# Common average reference
raw_orig = raw.copy()
raw = raw.set_eeg_reference('average', projection=False)

if plotGraphs:
    fig = raw_orig.plot(title='Before Re-referencing', n_channels=16, scalings=scalings)
    fig = raw.plot(title='After Re-referencing', n_channels=16, scalings=scalings)

EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.


In [11]:
# Bandpass filter data
raw_orig = raw.copy()
raw = raw.filter(l_freq=bp_l_freq, h_freq=bp_h_freq)

if plotGraphs:
    fig = raw_orig.plot(title='Before Filtering', scalings=scalings, duration=plot_duration)
    fig = raw.plot(title='After Filtering', scalings=scalings, duration=plot_duration)

Filtering raw data in 5 contiguous segments
Setting up band-pass filter from 1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 413 samples (3.304 sec)



### ICA Artifact Removal

In [12]:
if performICA:
    print('Performing ICA artifact removal...')
    raw_orig = raw.copy()

    # filter data to remove slow drifts
    raw_filt = raw.copy()
    raw_filt.filter(l_freq=1., h_freq=None)

    # ICA decomposition
    ica = mne.preprocessing.ICA(n_components=16, method='fastica', max_iter=200, random_state=42, verbose=True)
    ica = ica.fit(raw_filt)

In [13]:
# Plot ICA sources
if performICA and plotGraphs:
    fig = ica.plot_sources(raw_orig)

In [14]:
# Select source that corresponds to artifact and remove it
if performICA:
    ica.exclude = [0]
    # ica.exclude = [2]
    print('ICA sources to exclude: ', ica.exclude)

In [15]:
if performICA:
    ica.apply(raw)
    if plotGraphs:
        fig = raw_orig.plot(title='Before ICA', scalings=scalings, duration=plot_duration)
        fig = raw.plot(title='After ICA', scalings=scalings, duration=plot_duration)

### Epoch data

In [16]:
# Epoch data
events, event_id = mne.events_from_annotations(raw, event_id=event_dict)
epoch = mne.Epochs(raw, events, event_id=event_id, tmin=tmin, tmax=tmax, baseline=None, picks='eeg', preload=True)
print(epoch)

Used Annotations descriptions: ['MI/hands', 'rest']
Not setting metadata
Not setting metadata
100 matching events found
No baseline correction applied
0 projection items activated
Loading data for 100 events and 626 original time points ...
0 bad epochs dropped
<Epochs |  100 events (all good), 0 - 5 sec, baseline off, ~7.7 MB, data loaded,
 'MI/hands': 50
 'rest': 50>


### Features

In [17]:
# Labels
y = epoch.events[:,-1] - min(epoch.events[:,-1])

In [18]:
plt.close('all')
# Time-Domain Features
if features == 'time':
    scaler = Scaler(epoch.info)
    X = scaler.fit_transform(epoch.get_data())
    
    if ('P300' in task) and plotGraphs:
        fig = epoch['target'].average().plot_joint(times=[-0.2, 0., 0.3, 0.7], ts_args=dict(ylim=dict(eeg=[-10, 10])))
        fig = epoch['nontarget'].average().plot_joint(times=[-0.2, 0., 0.3, 0.7], ts_args=dict(ylim=dict(eeg=[-10, 10])))
#         fig = epoch['target'].average().plot_joint(times=[-0.2, 0., 0.3, 0.7], picks=['Fz','Cz','P3','Pz','P4','PO3','PO4','Oz'], ts_args=dict(ylim=dict(eeg=[-10, 10])))
#         fig = epoch['nontarget'].average().plot_joint(times=[-0.2, 0., 0.3, 0.7], picks=['Fz','Cz','P3','Pz','P4','PO3','PO4','Oz'], ts_args=dict(ylim=dict(eeg=[-10, 10])))

In [19]:
# Frequency-Domain Features
if features == 'psd':
    psds, freqs = psd_welch(epoch, average='mean', fmin=bp_l_freq, fmax=bp_h_freq, n_jobs=-1)
    X = 10 * np.log10(psds)
#     X = psds / np.sum(psds, axis=-1, keepdims=True)
    
    if ('MI' in task) and plotGraphs:
        sel_chs = [2, 3, 4, 5, 6, 7, 14, 15]
        psd_means_class_0 = np.transpose(np.mean(X[y==0], axis=0))
        psd_means_class_1 = np.transpose(np.mean(X[y==1], axis=0))
        psd_means_class_0 = psd_means_class_0[:,sel_chs]
        psd_means_class_1 = psd_means_class_1[:,sel_chs]
        
        fig = plt.figure()
        ax = fig.add_subplot(111)
        for i in range(len(sel_chs)):
            line = ax.plot(freqs, psd_means_class_0[:,i], ':', label=ch_names[sel_chs[i]] + ' Rest')
            ax.plot(freqs, psd_means_class_1[:,i], '-', label=ch_names[sel_chs[i]] + ' MI-hands', color=line[0].get_color())
        ax.set(title='Welch PSD', xlabel='Frequency (Hz)', ylabel='Power Spectral Density (dB)')
        ax.set_ylim(bottom=-135, top=-85)
        ax.legend(loc='best')

In [20]:
# Time-frequency features
if features =='tfr':
    # Parameters
    freqs = np.logspace(*np.log10([6, 35]), num=8)
#     freqs = np.linspace(2, 40, 20)
    print('TFR freqs: ', freqs)
    n_cycles = freqs / 2.
    time_bandwidth = 4.0 # param for multitaper
    
    # Compute TFR Power
    if tfr_type == 'morlet':
        power = tfr_morlet(epoch, freqs, n_cycles=n_cycles, use_fft=False, return_itc=False, average=False, n_jobs=-1)
    elif tfr_type == 'multitaper':
        power = tfr_multitaper(epoch, freqs, n_cycles=n_cycles, time_bandwidth=time_bandwidth, use_fft=False, return_itc=False, average=False, n_jobs=-1)
    print(power.data.shape)
    X = power.data
    
    if ('MI' in task) and plotGraphs:
        fig = power['rest'].average().plot_topo(baseline=baseline, mode='percent', cmap='jet', tmin=tmin, tmax=tmax, vmin=vmin, vmax=vmax, title='Average power (Rest)')
        fig = power['MI/hands'].average().plot_topo(baseline=baseline, mode='percent', cmap='jet', tmin=tmin, tmax=tmax, vmin=vmin, vmax=vmax, title='Average power (MI-hands)')

TFR freqs:  [ 6.          7.71912254  9.93080879 12.77618833 16.43682721 21.1463139
 27.2051647  35.        ]


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.


Not setting metadata
(100, 16, 8, 626)


[Parallel(n_jobs=4)]: Done  16 out of  16 | elapsed:    1.8s finished


In [21]:
# # plt.close('all')
# baseline = (0., 0.1)
# vmin, vmax = -1, 1
# fig = power['rest'].average().plot_topo(baseline=baseline, mode='percent', tmin=tmin, tmax=tmax, vmin=vmin, vmax=vmax, title='Average power (Rest)')
# fig = power['MI/hands'].average().plot_topo(baseline=baseline, mode='percent', tmin=tmin, tmax=tmax, vmin=vmin, vmax=vmax, title='Average power (MI-hands)')

In [22]:
# # Select Channels
# print('Selecting channels from data...')
# print('Original X.shape: ', X.shape)
# X = X[:,2:, :]
# print('X.shape: ', X.shape)

In [23]:
# Vectorize features
if len(X.shape) > 2:
    print('Vectorizing features to 2D...')
    print('Original X.shape: ', X.shape)
    vec = Vectorizer()
    X = vec.fit_transform(X)

print('X.shape: ', X.shape)
print('y.shape: ', y.shape)

Vectorizing features to 2D...
Original X.shape:  (100, 16, 8, 626)
X.shape:  (100, 80128)
y.shape:  (100,)


### Classification

In [24]:
# Set up cross validation
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=5, random_state=42)

# Set up scoring
scoring = 'accuracy'
scores = {'Classifier': [],
          'Score': [],
          'Std': []
         }

In [25]:
# Set up Classifiers
classifiers = []

# KNN
params = {}
params['n_neighbors'] = np.arange(2,11,1)
params['weights'] = ['uniform', 'distance']
clf = HalvingGridSearchCV(KNeighborsClassifier(), param_grid=params, n_jobs=-1, cv=cv, scoring=scoring)
classifiers.append(['KNN', clf, params])

# DT
params = {}
params['criterion'] = ['gini', 'entropy']
params['min_samples_split'] = np.arange(2,11,2)
clf = HalvingGridSearchCV(DecisionTreeClassifier(), param_grid=params, n_jobs=-1, cv=cv, scoring=scoring)
classifiers.append(['DT', clf, params])

# RF
params = {}
params['criterion'] = ['gini', 'entropy']
params['n_estimators'] = (10, 20, 30)
params['min_samples_split'] = np.arange(2,11,2)
clf = HalvingGridSearchCV(RandomForestClassifier(), param_grid=params, n_jobs=-1, cv=cv, scoring=scoring)
classifiers.append(['RF', clf, params])

# LDA
params = {}
params['solver'] = ['svd']
clf = HalvingGridSearchCV(LinearDiscriminantAnalysis(), param_grid=params, n_jobs=-1, cv=cv, scoring=scoring)
classifiers.append(['LDA', clf, params])

# SVM
params = {}
params['C'] = (1e-4, 1e-2, 1)
params['gamma'] = (1e-4, 1e-2, 1, 10)
params['kernel'] = ['linear', 'rbf']
clf = HalvingGridSearchCV(SVC(), param_grid=params, n_jobs=-1, cv=cv, scoring=scoring)
classifiers.append(['SVM', clf, params])

# SGD
params = {}
params['loss'] = ['hinge', 'log', 'modified_huber', 'squared_hinge', 'perceptron']
params['penalty'] = ['l2', 'l1', 'elasticnet']
params['alpha'] = (1e-4, 1e-2, 1, 10)
clf = HalvingGridSearchCV(SGDClassifier(), param_grid=params, n_jobs=-1, cv=cv, scoring=scoring)
classifiers.append(['SGD', clf, params])

In [26]:
# Train Classifiers
for c in range(len(classifiers)):
    clf_name = classifiers[c][0]
    clf = classifiers[c][1].fit(X, y)
    print("Training %s..." % clf_name)
    print('%s score: %2.2f' % (clf_name, clf.best_score_))
    print('%s std  : %2.2f' % (clf_name, np.mean(clf.cv_results_['std_test_score'])))
    print()
    scores['Classifier'].append(clf_name)
    scores['Score'].append(clf.best_score_)
    scores['Std'].append(np.mean(clf.cv_results_['std_test_score']))

Training KNN...
KNN score: 0.66
KNN std  : 0.07

Training DT...
DT score: 0.51
DT std  : 0.04

Training RF...
RF score: 0.51
RF std  : 0.04

Training LDA...
LDA score: 0.48
LDA std  : 0.07

Training SVM...
SVM score: 0.51
SVM std  : 0.00

Training SGD...
SGD score: 0.50
SGD std  : 0.00



In [27]:
# Score summary
df = pd.DataFrame(scores)
df

,Classifier,Score,Std
0,KNN,0.656,0.067729
1,DT,0.506,0.036331
2,RF,0.508,0.038524
3,LDA,0.484,0.068877
4,SVM,0.508,0.003627
5,SGD,0.500,0.001196


In [28]:
# Best Classifier
print('Best Classifier:')
df.loc[df['Score'].idxmax()]

Best Classifier:


Classifier         KNN
Score            0.656
Std           0.067729
Name: 0, dtype: object